# Web scraper for BBG site

With the help of CHatGPT, chat:

https://chat.openai.com/c/2230ca68-e38b-431c-8eb3-00d84acc69ef

BBG_scraping


## Functions

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd  # Import the pandas library
import re


def filter_func(tag):
    return tag.has_attr('id') and tag['id'].startswith('row_')

def fetch_page_content(url, headers):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.content
    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

def parse_html(html_content):
    return BeautifulSoup(html_content, "html.parser")

def extract_game_data(soup, url, rank):
    game_data = {}
    objectid = url.split('/')[-2]

    title_tag = soup.find('title')
    title_text = title_tag.text.split('|')[0] if title_tag else 'Unknown'
    game_data['Title'] = title_text

    json_data_match = re.search(r'GEEK\.geekitemPreload = ({.*?});', str(soup))
    if json_data_match:
        try:
            json_data = json.loads(json_data_match.group(1))
            game_data['Link'] = url
            ## Exploration of data follows Ranking
            game_data['Rank'] = rank  # Store in dictionary 
            # Extract the publishing year
            year_published = json_data.get('item', {}).get('yearpublished', 'Unknown')
            game_data['Year'] = year_published  # Store in dictionary        
            # Extract additional game info
            stats = json_data.get('item', {}).get('stats', {})
            game_data['UsersRated'] = stats.get('usersrated', 'Unknown')
            game_data['AveRating'] = stats.get('average', 'Unknown')
            game_data['Baverage'] = stats.get('baverage', 'Unknown')
            game_data['StdDev'] = stats.get('stddev', 'Unknown')
            game_data['AveWeight'] = stats.get('avgweight', 'Unknown')
            game_data['NumWeights'] = stats.get('numweights', 'Unknown')
            game_data['NumPlays'] = stats.get('numplays', 'Unknown')
            linkcounts = json_data.get('item', {}).get('linkcounts', {})
            game_data['NumAwards'] = linkcounts.get('boardgamehonor', 0)
            polls = json_data.get('item', {}).get('polls', {})
            #print( polls.get('userplayers', 'Unknown')['totalvotes'] )
            game_data['PCountRecomm_min'] = polls.get('userplayers', 'Unknown')['recommended'][0]['min']
            game_data['PCountRecomm_max'] = polls.get('userplayers', 'Unknown')['recommended'][0]['max']
            game_data['PCountBest_min'] = polls.get('userplayers', 'Unknown')['best'][0]['min']
            game_data['PCountBest_max'] = polls.get('userplayers', 'Unknown')['best'][0]['max']
            game_data['PCount_votes'] = polls.get('userplayers', 'Unknown')['totalvotes']
            game_data['MinPlayTime'] = json_data.get('item', {}).get('minplaytime', 'NA')
            game_data['MaxPlayTime'] = json_data.get('item', {}).get('maxplaytime', 'NA')
            mechanics = json_data.get('item', {}).get('links', {}).get('boardgamemechanic', [])
            mechstring = ''
            for i, mech in enumerate(mechanics):
                mechstring += mech['name']
                if i < len(mech)-1:
                    mechstring += '\n'
            game_data['Mechanics'] = mechstring          
            rankinfo = json_data.get('item', {}).get('rankinfo', [])
            for i, rinf in enumerate(rankinfo):
                if rinf['prettyname'] != 'Board Game Rank':
                    game_data['GameCategory_'+str(i)] = rinf['veryshortprettyname']
                    game_data['RankCategory_'+str(i)] = rinf['rank']
                    game_data['ScoreCategory_'+str(i)] = rinf['baverage']          
            rank += 1
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
    else:
        print("JSON data not found in HTML content.")

    # Amazon price extraction logic
    amazon_url = "https://api.geekdo.com/api/amazon/textads"
    params = {"locale": "us", "objectid": objectid, "objecttype": "thing"}
    response = requests.get(amazon_url, params=params)
    if response.status_code == 200:
        amazon_data = response.json()
        if isinstance(amazon_data, dict):  # Check if the response is a dictionary
            price = amazon_data.get('us', {}).get('newprice', 'Unknown')
        else:
            price = 'Unknown'
        game_data['AmazonPrice'] = price
    else:
        game_data['AmazonPrice'] = 'Unknown'

    return game_data

def scrape_boardgame_data(url, headers, rank):
    html_content = fetch_page_content(url, headers)
    if html_content:
        soup = parse_html(html_content)
        return extract_game_data(soup, url, rank)
    else:
        return None




## Scrapping for the BGG titles and webpages (This works for 1000 games ONLY)

In [1]:
import requests
from bs4 import BeautifulSoup
import time


# Initialize an empty list to store the extracted URLs
listurls = []

# Define headers to mimic a real browser request
headers = {'User-Agent': 'Mozilla/6.0'}

# Loop through pages 1 to X
for page_num in range(1, 25):
    URL = f"https://boardgamegeek.com/browse/boardgame/page/{page_num}"
    page = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")

    print(f"HTTP Status Code for page {page_num}: {page.status_code}")

    # Print a snippet of the page content for debugging
    #print("Page content snippet:", soup.text[:500])
    #print(f"HTTP Status Code for page {page_num}: {page.status_code}")

    # Find all instances of elements with id starting with 'row_'
    elements = soup.find_all(filter_func)

    # Loop through each element and find the <a> tags
    for element in elements:
        a_tags = element.find_all('a')
        for a_tag in a_tags:
            # Check if the href attribute exists and follows the specific structure
            if a_tag.has_attr('href') and '/boardgame/' in a_tag['href']:
                url = f"https://boardgamegeek.com{a_tag['href']}"
                # Check for duplicates before appending
                if url not in listurls:
                    listurls.append(url)
    
    # Print the current status
    print(f"Scraping page {page_num} - Total URLs collected: {len(listurls)}")
    
    # Optional: Sleep for a few seconds between requests to avoid overloading the server
    #time.sleep(2)

# Print the total number of extracted URLs
print(f"Total URLs extracted: {len(listurls)}")


HTTP Status Code for page 1: 200
Scraping page 1 - Total URLs collected: 101
HTTP Status Code for page 2: 200
Scraping page 2 - Total URLs collected: 201
HTTP Status Code for page 3: 200
Scraping page 3 - Total URLs collected: 302
HTTP Status Code for page 4: 200
Scraping page 4 - Total URLs collected: 403
HTTP Status Code for page 5: 200
Scraping page 5 - Total URLs collected: 503
HTTP Status Code for page 6: 200
Scraping page 6 - Total URLs collected: 603
HTTP Status Code for page 7: 200
Scraping page 7 - Total URLs collected: 703
HTTP Status Code for page 8: 200
Scraping page 8 - Total URLs collected: 803
HTTP Status Code for page 9: 200
Scraping page 9 - Total URLs collected: 903
HTTP Status Code for page 10: 200
Scraping page 10 - Total URLs collected: 1004
HTTP Status Code for page 11: 200
Scraping page 11 - Total URLs collected: 1004
HTTP Status Code for page 12: 200
Scraping page 12 - Total URLs collected: 1004
HTTP Status Code for page 13: 200
Scraping page 13 - Total URLs col

## Alternative 2 to get URLs - BGG Ranking Historicals

In [14]:
## Getting the list of URLS from the official CSV 
## https://gitlab.com/recommend.games/bgg-ranking-historicals/-/tree/master?ref_type=heads

import pandas as pd

df = pd.read_csv('FullBGGData_2024-05-08.csv', sep=',', header=0)
df.head()

,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
0,224517,Brass: Birmingham,2018,1,8.60,8.416,44997,/boardgame/224517/brass-birmingham,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...
1,161936,Pandemic Legacy: Season 1,2015,2,8.53,8.382,53188,/boardgame/161936/pandemic-legacy-season-1,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...
2,174430,Gloomhaven,2017,3,8.59,8.357,61809,/boardgame/174430/gloomhaven,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...
3,342942,Ark Nova,2021,4,8.54,8.333,42308,/boardgame/342942/ark-nova,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.61,8.243,23460,/boardgame/233078/twilight-imperium-fourth-edi...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...


In [15]:
baseurl = 'https://boardgamegeek.com'
listurls = df.URL.to_list()
listurls = [baseurl + url for url in listurls]
len(listurls)

26286

## TEST

In [7]:
headers = {'User-Agent': 'Mozilla/6.0'}

credits = listurls[0]+'/credits#boardgamemechanic'
html_content = fetch_page_content(credits, headers)

if html_content:
        soup = parse_html(html_content)
json_data_match = re.search(r'GEEK\.geekitemPreload = ({.*?});', str(soup))
json_data = json.loads(json_data_match.group(1))
mechanics = json_data.get('item', {}).get('links', {}).get('boardgamemechanic', [])
print(mechanics)
soup
#urlcredits = listurls[0]+'/credits'
#print(urlcredits)
#html_content = fetch_page_content(urlcredits, headers)
#soup = parse_html(html_content)
#soup

[{'name': 'Hand Management', 'sortindex': '1', 'objecttype': 'property', 'objectid': '2040', 'primarylink': 0, 'itemstate': 'approved', 'href': '/boardgamemechanic/2040/hand-management', 'canonical_link': 'https://boardgamegeek.com/boardgamemechanic/2040/hand-management'}, {'name': 'Income', 'sortindex': '1', 'objecttype': 'property', 'objectid': '2902', 'primarylink': 0, 'itemstate': 'approved', 'href': '/boardgamemechanic/2902/income', 'canonical_link': 'https://boardgamegeek.com/boardgamemechanic/2902/income'}, {'name': 'Loans', 'sortindex': '1', 'objecttype': 'property', 'objectid': '2904', 'primarylink': 0, 'itemstate': 'approved', 'href': '/boardgamemechanic/2904/loans', 'canonical_link': 'https://boardgamegeek.com/boardgamemechanic/2904/loans'}, {'name': 'Market', 'sortindex': '1', 'objecttype': 'property', 'objectid': '2900', 'primarylink': 0, 'itemstate': 'approved', 'href': '/boardgamemechanic/2900/market', 'canonical_link': 'https://boardgamegeek.com/boardgamemechanic/2900/m

<!DOCTYPE html>

<html lang="en-US" ng-app="GeekApp" ng-cloak="">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" id="vp" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Brass: Birmingham | Board Game | BoardGameGeek</title>
<link href="https://boardgamegeek.com/boardgame/224517/brass-birmingham" rel="canonical"/>
<link href="https://cf.geekdo-static.com/icons/touch-icon180.png" rel="apple-touch-icon"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="icon" type="image/ico"/>
<link href="/game-opensearch.xml" rel="search" title="BGG Game Search" type="application/opensearchdescription+xml"/>
<meta content="BGG" name="apple-mobile-web-app-title"/>
<meta content="#2e2b47" name="theme-color"/>
<link href="https://api.geekdo.com" rel="precon

In [15]:
len(listurls)

26286

In [18]:
data_list = []
rank = 1
for i, url in enumerate(listurls):
    if i < 3000:
        game_data = scrape_boardgame_data(url, headers, rank)
        if game_data:
            data_list.append(game_data)
            rank += 1
df = pd.DataFrame(data_list)
df.head()

KeyboardInterrupt: 

In [20]:
df = pd.DataFrame(data_list)
df

,Title,Link,Rank,Year,UsersRated,AveRating,Baverage,StdDev,AveWeight,NumWeights,...,GameCategory_1,RankCategory_1,ScoreCategory_1,AmazonPrice,GameCategory_2,RankCategory_2,ScoreCategory_2,GameCategory_3,RankCategory_3,ScoreCategory_3
0,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,1,2018,45176,8.59882,8.41597,1.41467,3.8814,2184,...,Strategy,1,8.43294,$70.45,NaN,NaN,NaN,NaN,NaN,NaN
1,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,2,2015,53263,8.52865,8.38135,1.5974,2.8308,1448,...,Thematic,1,8.3821,$71.98,Strategy,2,8.36925,NaN,NaN,NaN
2,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,3,2017,61887,8.59213,8.3562,1.73926,3.9099,2553,...,Thematic,2,8.33583,$300.99,Strategy,4,8.31822,NaN,NaN,NaN
3,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,4,2021,42599,8.53609,8.33362,1.38106,3.7568,2237,...,Strategy,3,8.34558,$62.82,NaN,NaN,NaN,NaN,NaN,NaN
4,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,5,2017,23551,8.60338,8.2422,1.61956,4.3125,1152,...,Thematic,3,8.26259,$131.99,Strategy,5,8.26533,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,Autobahn,https://boardgamegeek.com/boardgame/348554/aut...,2294,2022,1468,7.09098,6.1301,1.45403,3.8252,103,...,Strategy,1090,6.268,$51.01,NaN,NaN,NaN,NaN,NaN,NaN
2294,GoodCritters,https://boardgamegeek.com/boardgame/255615/goo...,2295,2018,1545,7.01845,6.13193,1.30045,1.4545,33,...,Party,104,6.52474,$20.89,NaN,NaN,NaN,NaN,NaN,NaN
2295,We the People,https://boardgamegeek.com/boardgame/620/we-people,2296,1993,1148,7.30502,6.12896,1.44088,2.7034,145,...,War,172,6.96794,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
2296,Founders of Gloomhaven,https://boardgamegeek.com/boardgame/214032/fou...,2297,2018,3136,6.60502,6.12989,1.73012,4.1122,205,...,Strategy,1162,6.2101,$49.95,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import pandas as pd  # Import the pandas library

# Initialize an empty list to store the data
data_list = []

# Sample list of URLs (replace with your actual list of 2003 URLs)
#listurls = ['https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1']

# Sample headers (replace with your actual headers)
headers = {'User-Agent': 'Mozilla/5.0'}

rank = 1

sublisturls = listurls[:5000]

for url in sublisturls:
    # Initialize a dictionary to store the data for each game
    game_data = {}
    
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")

    # Extract objectid from the URL
    objectid = url.split('/')[-2]

    # Title
    title_tag = soup.find('title')
    title_text = title_tag.text.split('|')[0] if title_tag else 'Unknown'
    game_data['Title'] = title_text  # Store in dictionary
    
    # Year published
    json_data_match = re.search(r'GEEK\.geekitemPreload = ({.*?});', str(soup))
    
    # Parse the JSON data if found
    if json_data_match:
        json_data_str = json_data_match.group(1)
        
        try:
            json_data = json.loads(json_data_str)
            
            game_data['Link'] = url

            ## Exploration of data follows Ranking
            game_data['Rank'] = rank  # Store in dictionary 

            # Extract the publishing year
            year_published = json_data.get('item', {}).get('yearpublished', 'Unknown')
            game_data['Year'] = year_published  # Store in dictionary
            
            # Extract additional game info
            stats = json_data.get('item', {}).get('stats', {})
            game_data['UsersRated'] = stats.get('usersrated', 'Unknown')
            game_data['AveRating'] = stats.get('average', 'Unknown')
            game_data['Baverage'] = stats.get('baverage', 'Unknown')
            game_data['StdDev'] = stats.get('stddev', 'Unknown')
            game_data['AveWeight'] = stats.get('avgweight', 'Unknown')
            game_data['NumWeights'] = stats.get('numweights', 'Unknown')
            game_data['NumPlays'] = stats.get('numplays', 'Unknown')
            
            linkcounts = json_data.get('item', {}).get('linkcounts', {})
            game_data['NumAwards'] = linkcounts.get('boardgamehonor', 0)

            polls = json_data.get('item', {}).get('polls', {})
            #print( polls.get('userplayers', 'Unknown')['totalvotes'] )
            game_data['PCountRecomm_min'] = polls.get('userplayers', 'Unknown')['recommended'][0]['min']
            game_data['PCountRecomm_max'] = polls.get('userplayers', 'Unknown')['recommended'][0]['max']
            game_data['PCountBest_min'] = polls.get('userplayers', 'Unknown')['best'][0]['min']
            game_data['PCountBest_max'] = polls.get('userplayers', 'Unknown')['best'][0]['max']
            game_data['PCount_votes'] = polls.get('userplayers', 'Unknown')['totalvotes']

            game_data['MinPlayTime'] = json_data.get('item', {}).get('minplaytime', 'NA')
            game_data['MaxPlayTime'] = json_data.get('item', {}).get('maxplaytime', 'NA')

            mechanics = json_data.get('item', {}).get('links', {}).get('boardgamemechanic', [])
            mechstring = ''
            for i, mech in enumerate(mechanics):
                mechstring += mech['name']
                if i < len(mech)-1:
                    mechstring += '\n'
            game_data['Mechanics'] = mechstring
            
            rankinfo = json_data.get('item', {}).get('rankinfo', [])
            for i, rinf in enumerate(rankinfo):
                if rinf['prettyname'] != 'Board Game Rank':
                    game_data['Category_'+str(i)] = rinf['veryshortprettyname']
                    game_data['RankCat_'+str(i)] = rinf['rank']
                    game_data['ScoreCat_'+str(i)] = rinf['baverage']
            
            rank += 1
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
    else:
        print("JSON data not found in HTML content.")
    
    ############## AMAZON price ##############
    # Define the base URL and parameters for Amazon price
    amazon_url = "https://api.geekdo.com/api/amazon/textads"
    params = {
        "locale": "us",
        "objectid": objectid,
        "objecttype": "thing"
    }
    # Make a request to the Amazon API endpoint
    response = requests.get(amazon_url, params=params)
    if response.status_code == 200:
        amazon_data = response.json()
        if isinstance(amazon_data, dict):  # Check if the response is a dictionary
            price = amazon_data.get('us', {}).get('newprice', 'Unknown')
        else:
            price = 'Unknown'
        game_data['AmazonPrice'] = price

    else:
        game_data['AmazonPrice'] = 'Unknown'
    ############################################

    # Append the game data to the list
    data_list.append(game_data)

    # Uncomment the following line to exit the loop for testing
    # sys.exit()

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [22]:
data_list

[{'Title': 'Brass: Birmingham ',
  'Link': 'https://boardgamegeek.com/boardgame/224517/brass-birmingham',
  'Rank': 1,
  'Year': '2018',
  'UsersRated': '46043',
  'AveRating': '8.59731',
  'Baverage': '8.41456',
  'StdDev': '1.41606',
  'AveWeight': '3.8789',
  'NumWeights': '2229',
  'NumPlays': '124812',
  'NumAwards': 20,
  'PCountRecomm_min': 2,
  'PCountRecomm_max': 4,
  'PCountBest_min': 3,
  'PCountBest_max': 4,
  'PCount_votes': '1106',
  'MinPlayTime': '60',
  'MaxPlayTime': '120',
  'Mechanics': 'Hand Management\nIncome\nLoans\nMarket\nNetwork and Route Building\nTags\n',
  'Category_1': 'Strategy ',
  'RankCat_1': '1',
  'ScoreCat_1': '8.43113',
  'AmazonPrice': '$63.48'},
 {'Title': 'Pandemic Legacy: Season 1 ',
  'Link': 'https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1',
  'Rank': 2,
  'Year': '2015',
  'UsersRated': '53558',
  'AveRating': '8.52713',
  'Baverage': '8.37875',
  'StdDev': '1.5984',
  'AveWeight': '2.8316',
  'NumWeights': '1461',
  'Num

In [21]:
# Save the DataFrame to a CSV file
df.to_csv('bgg_data_2300_2024-05.csv', index=False)

In [37]:
import requests

# Define the base URL and parameters
base_url = "https://api.geekdo.com/api/amazon"
endpoints = ["textads", "itemurls"]
params = {
    "locale": "us",
    "objectid": "224517",
    "objecttype": "thing"
}

# Loop over the endpoints and make requests
for endpoint in endpoints:
    url = f"{base_url}/{endpoint}"
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        print(f"Data from {endpoint}:")
        print(data)  # Print the parsed JSON, or extract specific data as needed
    else:
        print(f"Failed to fetch data from {endpoint}. Server responded with status code {response.status_code}")



Data from textads:
{'us': {'sponsorname': 'Amazon', 'supersaver': False, 'prime': False, 'title': 'Get Brass: Birmingham on Amazon $79.95', 'defaultprice': '$79.95', 'newprice': '$79.95', 'listprice': None, 'url': 'https://www.amazon.com/dp/1988884047?tag=itemtext-boardgamegeek-20&linkCode=ogi&th=1&psc=1', 'sponsorurl': 'https://www.amazon.com/dp/1988884047?tag=itemtext-boardgamegeek-20&linkCode=ogi&th=1&psc=1', 'add_to_cart_url': 'https://www.amazon.com/gp/aws/cart/add.html?AssociateTag=itemtext-boardgamegeek-20&ASIN.1=1988884047'}}
Data from itemurls:
{'us': '//ws-na.amazon-adsystem.com/widgets/q?ServiceVersion=20070822&OneJS=1&Operation=GetAdHtml&MarketPlace=US&source=ac&ref=tf_til&ad_type=product_link&tracking_id=boardgamegeek-20&marketplace=amazon&region=US&placement=1988884047&asins=1988884047&show_border=false&link_opens_in_new_window=true'}


In [16]:
listurls.index(url)

1014

In [23]:
# Initialize an empty list to store the data
#data_list = []

# Sample list of URLs (replace with your actual list of 2003 URLs)
#listurls = ['https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1']

lasturl = url

# Sample headers (replace with your actual headers)
headers = {'User-Agent': 'Mozilla/5.0'}

initiate = False
for url in listurls:
    if url == lasturl:
        initiate = True
    
    if initiate:
        # Initialize a dictionary to store the data for each game
        game_data = {}
        
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")

        # Extract objectid from the URL
        objectid = url.split('/')[-2]

        # Title
        title_tag = soup.find('title')
        title_text = title_tag.text.split('|')[0] if title_tag else 'Unknown'
        game_data['Title'] = title_text  # Store in dictionary
        
        # Year published
        json_data_match = re.search(r'GEEK\.geekitemPreload = ({.*?});', str(soup))
        
        # Parse the JSON data if found
        if json_data_match:
            json_data_str = json_data_match.group(1)
            
            try:
                json_data = json.loads(json_data_str)
                
                game_data['Link'] = url

                ## Exploration of data follows Ranking
                game_data['Rank'] = rank  # Store in dictionary 

                # Extract the publishing year
                year_published = json_data.get('item', {}).get('yearpublished', 'Unknown')
                game_data['Year'] = year_published  # Store in dictionary
                
                # Extract additional game info
                stats = json_data.get('item', {}).get('stats', {})
                game_data['UsersRated'] = stats.get('usersrated', 'Unknown')
                game_data['AveRating'] = stats.get('average', 'Unknown')
                game_data['Baverage'] = stats.get('baverage', 'Unknown')
                game_data['StdDev'] = stats.get('stddev', 'Unknown')
                game_data['AveWeight'] = stats.get('avgweight', 'Unknown')
                game_data['NumWeights'] = stats.get('numweights', 'Unknown')
                game_data['NumPlays'] = stats.get('numplays', 'Unknown')
                
                linkcounts = json_data.get('item', {}).get('linkcounts', {})
                game_data['NumAwards'] = linkcounts.get('boardgamehonor', 0)

                polls = json_data.get('item', {}).get('polls', {})
                #print( polls.get('userplayers', 'Unknown')['totalvotes'] )
                game_data['PCountRecomm_min'] = polls.get('userplayers', 'Unknown')['recommended'][0]['min']
                game_data['PCountRecomm_max'] = polls.get('userplayers', 'Unknown')['recommended'][0]['max']
                game_data['PCountBest_min'] = polls.get('userplayers', 'Unknown')['best'][0]['min']
                game_data['PCountBest_max'] = polls.get('userplayers', 'Unknown')['best'][0]['max']
                game_data['PCount_votes'] = polls.get('userplayers', 'Unknown')['totalvotes']

                game_data['MinPlayTime'] = json_data.get('item', {}).get('minplaytime', 'NA')
                game_data['MaxPlayTime'] = json_data.get('item', {}).get('maxplaytime', 'NA')

                mechanics = json_data.get('item', {}).get('links', {}).get('boardgamemechanic', [])
                mechstring = ''
                for i, mech in enumerate(mechanics):
                    mechstring += mech['name']
                    if i < len(mech)-1:
                        mechstring += '\n'
                game_data['Mechanics'] = mechstring
                
                rankinfo = json_data.get('item', {}).get('rankinfo', [])
                for i, rinf in enumerate(rankinfo):
                    if rinf['prettyname'] != 'Board Game Rank':
                        game_data['Category_'+str(i)] = rinf['veryshortprettyname']
                        game_data['RankCat_'+str(i)] = rinf['rank']
                        game_data['ScoreCat_'+str(i)] = rinf['baverage']
                
                rank += 1
            except json.JSONDecodeError as e:
                print(f"JSONDecodeError: {e}")
        else:
            print("JSON data not found in HTML content.")
        
        ############## AMAZON price ##############
        # Define the base URL and parameters for Amazon price
        amazon_url = "https://api.geekdo.com/api/amazon/textads"
        params = {
            "locale": "us",
            "objectid": objectid,
            "objecttype": "thing"
        }
        # Make a request to the Amazon API endpoint
        response = requests.get(amazon_url, params=params)
        if response.status_code == 200:
            amazon_data = response.json()
            if isinstance(amazon_data, dict):  # Check if the response is a dictionary
                price = amazon_data.get('us', {}).get('newprice', 'Unknown')
            else:
                price = 'Unknown'
            game_data['AmazonPrice'] = price

        else:
            game_data['AmazonPrice'] = 'Unknown'
        ############################################

        # Append the game data to the list
        data_list.append(game_data)

    # Uncomment the following line to exit the loop for testing
    # sys.exit()

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

IndexError: list index out of range

In [42]:
df

,Title,Link,Rank,Year,UsersRated,AveRating,Baverage,StdDev,AveWeight,NumWeights,...,Category_1,RankCat_1,ScoreCat_1,AmazonPrice,Category_2,RankCat_2,ScoreCat_2,Category_3,RankCat_3,ScoreCat_3
0,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,1,2018,46043,8.59731,8.41456,1.41606,3.8789,2229,...,Strategy,1,8.43113,$63.48,NaN,NaN,NaN,NaN,NaN,NaN
1,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,2,2015,53558,8.52713,8.37875,1.5984,2.8316,1461,...,Thematic,1,8.37923,$71.97,Strategy,2,8.3668,NaN,NaN,NaN
2,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,3,2017,62246,8.58836,8.35218,1.73928,3.9115,2566,...,Thematic,2,8.33121,$285.49,Strategy,4,8.31405,NaN,NaN,NaN
3,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,4,2021,43695,8.53514,8.33503,1.38399,3.7606,2297,...,Strategy,3,8.3448,$56.92,NaN,NaN,NaN,NaN,NaN,NaN
4,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,5,2017,23849,8.60087,8.24171,1.62048,4.319,1166,...,Thematic,3,8.26231,$131.99,Strategy,5,8.26381,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,Whoowasit?,https://boardgamegeek.com/boardgame/33643/whoo...,3309,2007,1094,6.80182,5.92831,1.50822,1.4561,57,...,Children's,30,6.61902,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
3309,Small Islands,https://boardgamegeek.com/boardgame/236248/sma...,3310,2018,1380,6.62765,5.92564,1.34981,2.0455,44,...,Family,1066,6.01504,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
3310,The Blood of an Englishman,https://boardgamegeek.com/boardgame/206504/blo...,3311,2016,1496,6.54979,5.92681,1.23373,1.8261,23,...,Family,1050,6.0255,$16.19,NaN,NaN,NaN,NaN,NaN,NaN
3311,Panzer: The Game of Small Unit Actions and Com...,https://boardgamegeek.com/boardgame/85769/panz...,3312,2012,553,7.85953,5.92759,1.47996,3.7536,69,...,War,192,6.93608,Unknown,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df = pd.DataFrame(data_list)

In [29]:
name = f'bgg_data_0-{1000}.pkl'
print('Name of pickle file:')
print(name)

df.to_pickle(name)

Name of pickle file:
bgg_data_0-1000.pkl


In [41]:
(df
 .query('PCountBest_min > 4')
 .query('PCountBest_max < 7')
)

,Title,Link,Rank,Year,UsersRated,AveRating,Baverage,StdDev,AveWeight,NumWeights,...,Category_1,RankCat_1,ScoreCat_1,AmazonPrice,Category_2,RankCat_2,ScoreCat_2,Category_3,RankCat_3,ScoreCat_3
4,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,5,2017,23849,8.60087,8.24171,1.62048,4.319,1166,...,Thematic,3,8.26231,$131.99,Strategy,5,8.26381,NaN,NaN,NaN
41,Heat: Pedal to the Metal,https://boardgamegeek.com/boardgame/366013/hea...,42,2022,22984,8.07355,7.79997,1.31027,2.196,658,...,Family,4,7.83387,$74.99,NaN,NaN,NaN,NaN,NaN,NaN
91,El Grande,https://boardgamegeek.com/boardgame/93/el-grande,92,1995,29011,7.75581,7.57689,1.33082,2.9705,2342,...,Strategy,82,7.60759,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
94,Battlestar Galactica: The Board Game,https://boardgamegeek.com/boardgame/37111/batt...,95,2008,36538,7.73001,7.53881,1.59626,3.2499,2213,...,Thematic,32,7.54329,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
114,Twilight Imperium: Third Edition,https://boardgamegeek.com/boardgame/12493/twil...,115,2005,19287,7.79287,7.49565,1.71749,4.258,2085,...,Thematic,33,7.54016,$195.00,Strategy,113,7.50412,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,Formula Motor Racing,https://boardgamegeek.com/boardgame/635/formul...,3273,1995,2005,6.40748,5.93716,1.29191,1.1801,211,...,Family,1079,6.00725,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
3291,A Brief History of the World,https://boardgamegeek.com/boardgame/46007/brie...,3292,2009,948,6.8629,5.92966,1.43616,2.8681,91,...,Strategy,1436,6.04148,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
3292,Ghost Blitz: 5 to 12,https://boardgamegeek.com/boardgame/148532/gho...,3293,2013,1030,6.93647,5.93023,1.40416,1.303,33,...,Party,171,6.30006,Unknown,NaN,NaN,NaN,NaN,NaN,NaN
3305,Valparaíso,https://boardgamegeek.com/boardgame/257058/val...,3306,2018,952,6.87959,5.92737,1.12636,2.8049,41,...,Strategy,1450,6.0343,$48.95,NaN,NaN,NaN,NaN,NaN,NaN


,Title,Link,Rank,Year,UsersRated,AveRating,Baverage,StdDev,AveWeight,NumWeights,...,Category_1,RankCat_1,ScoreCat_1,AmazonPrice,Category_2,RankCat_2,ScoreCat_2,Category_3,RankCat_3,ScoreCat_3
